In [16]:
import PyPDF2
import fitz

In [21]:
doc = fitz.open("Qualcomm Report.pdf")

In [24]:
doc.metadata

{'format': 'PDF 1.4',
 'title': 'QCOM 9.27.20 10-K 2020',
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': 'Workiva',
 'producer': 'Wdesk Fidelity Content Translations Version 004.058.002',
 'creationDate': 'D:20201104181943Z',
 'modDate': "D:20201104110650-08'00'",
 'trapped': '',
 'encryption': None}

In [36]:
print(doc.pageCount)
print(doc.get_toc)
outlines = doc.getToC()
outlines

102
<bound method getToC of Document('Qualcomm Report.pdf')>


[]

In [37]:
page = doc.loadPage(4)

In [80]:
Page1 = page.getText()
Page1

'UNITED STATES\nSECURITIES AND EXCHANGE COMMISSION\nWASHINGTON, D.C. 20549\nFORM 10-K \n(Mark one)\n \n \n \n☒\n \nANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT \nOF 1934\nFor the fiscal year ended September 27, 2020 \nOR\n \n \n \n☐\n \nTRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE \nACT OF 1934\nFor the transition period from                      to                      .\nCommission File Number 0-19528 \nQUALCOMM Incorporated\n(Exact name of registrant as specified in its charter)\n \nDelaware\n95-3685934\n(State or Other Jurisdiction of Incorporation \nor Organization)\n(I.R.S. Employer\nIdentification No.)\n \n \n5775 Morehouse Dr., San Diego,\nCalifornia\n92121-1714\n(Address of Principal Executive Offices) \n(Zip Code)\n(858) 587-1121 \n(Registrant’s telephone number, including area code)\nSecurities registered pursuant to section 12(b) of the Act:\nTitle of Each Class\n Trading Symbol(s) \nName of Each Exchange on Wh

In [79]:
import pandas as pd
df = pd.DataFrame(Page1)

ValueError: DataFrame constructor not properly called!

In [45]:
def fonts(doc, granularity=False):
    """Extracts fonts and their usage in PDF documents.
    :param doc: PDF document to iterate through
    :type doc: <class 'fitz.fitz.Document'>
    :param granularity: also use 'font', 'flags' and 'color' to discriminate text
    :type granularity: bool
    :rtype: [(font_size, count), (font_size, count}], dict
    :return: most used fonts sorted by count, font style information
    """
    styles = {}
    font_counts = {}

    for page in doc:
        blocks = page.getText("Overview")["blocks"]
        for b in blocks:  # iterate through the text blocks
            if b['type'] == 0:  # block contains text
                for l in b["lines"]:  # iterate through the text lines
                    for s in l["spans"]:  # iterate through the text spans
                        if granularity:
                            identifier = "{0}_{1}_{2}_{3}".format(s['size'], s['flags'], s['font'], s['color'])
                            styles[identifier] = {'size': s['size'], 'flags': s['flags'], 'font': s['font'],
                                                  'color': s['color']}
                        else:
                            identifier = "{0}".format(s['size'])
                            styles[identifier] = {'size': s['size'], 'font': s['font']}

                        font_counts[identifier] = font_counts.get(identifier, 0) + 1  # count the fonts usage

    font_counts = sorted(font_counts.items(), key=itemgetter(1), reverse=True)

    if len(font_counts) < 1:
        raise ValueError("Zero discriminating fonts found!")

    return font_counts, styles

In [51]:
def headers_para(doc, size_tag):
    """Scrapes headers & paragraphs from PDF and return texts with element tags.
    :param doc: PDF document to iterate through
    :type doc: <class 'fitz.fitz.Document'>
    :param size_tag: textual element tags for each size
    :type size_tag: dict
    :rtype: list
    :return: texts with pre-prended element tags
    """
    headers_para = []  # list with headers and paragraphs
    first = True  # boolean operator for first header
    previous_s = {}  # previous span

    for page in doc:
        blocks = page.getText("dict")["blocks"]
        for b in blocks:  # iterate through the text blocks
            if b['type'] == 0:  # this block contains text

                # REMEMBER: multiple fonts and sizes are possible IN one block

                block_string = ""  # text found in block
                for l in b["lines"]:  # iterate through the text lines
                    for s in l["spans"]:  # iterate through the text spans
                        if s['text'].strip():  # removing whitespaces:
                            if first:
                                previous_s = s
                                first = False
                                block_string = size_tag[s['size']] + s['text']
                            else:
                                if s['size'] == previous_s['size']:

                                    if block_string and all((c == "|") for c in block_string):
                                        # block_string only contains pipes
                                        block_string = size_tag[s['size']] + s['text']
                                    if block_string == "":
                                        # new block has started, so append size tag
                                        block_string = size_tag[s['size']] + s['text']
                                    else:  # in the same block, so concatenate strings
                                        block_string += " " + s['text']

                                else:
                                    headers_para.append(block_string)
                                    block_string = size_tag[s['size']] + s['text']

                                previous_s = s

                    # new block started, indicating with a pipe
                    block_string += "|"

                headers_para.append(block_string)

    return headers_para

In [56]:
headers_para(page, )

TypeError: headers_para() missing 2 required positional arguments: 'doc' and 'size_tag'

In [54]:
def getpdfcontent(path):
    pdf_content = extract_pdf(path)
    text = re.sub(r"\n{2,}", "<131313>", pdf_content['content'])
    text = text.replace("\n", " ")
    text = text.replace("<131313>", "\n\n")
    paragraphs = extractparagraphs(text.splitlines())
    return removeduplicates(paragraphs)

In [73]:
import re
import textract
#read the content of pdf as text
text = textract.process('Qualcomm Report.pdf')
#use four space as paragraph delimiter to convert the text into list of paragraphs.


ShellError: The command `pdftotext Qualcomm Report.pdf -` failed with exit code 127
------------- stdout -------------
------------- stderr -------------


In [68]:
text

NameError: name 'text' is not defined